In [23]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import rioxarray
import platform
import yaml
from tqdm import tqdm 
from pathlib import Path
from rasterio.warp import transform
from pyproj import Transformer
from ascat.read_native.ragged_array_ts import CellFileCollection
from ascat.cell import CellGridFiles
from dask.diagnostics import ProgressBar


## Map Sentinel-1 and ASCAT

In [ ]:
def process_sentinel_and_ascat(zarr_path, sentinel_epsg, output_path="ascat_s1_collocated.nc"):
    """
    Load Sentinel-1 Zarr file, assign EPSG, collocate with ASCAT data, and save to NetCDF.
    
    Parameters:
        zarr_path (str or Path): Path to Sentinel-1 Zarr file
        sentinel_epsg (str): EPSG code for Sentinel-1 data (e.g., "EPSG:27704")
        output_path (str): Path to save the resulting NetCDF file
    """
    # Loading Sentinel-1
    print("Loading Sentinel-1 data...")
    sentinel_ds = xr.open_zarr(zarr_path)
    sentinel_ds = sentinel_ds.rio.write_crs(sentinel_epsg)
    print("Done")

    # Computing bounding box
    sentinel_ds_latlon = sentinel_ds.rio.reproject("EPSG:4326")
    latmin = float(sentinel_ds_latlon.y.min())
    latmax = float(sentinel_ds_latlon.y.max())
    lonmin = float(sentinel_ds_latlon.x.min())
    lonmax = float(sentinel_ds_latlon.x.max())
    bbox = (latmin, latmax, lonmin, lonmax)

    # Loading ASCAT with progress
    print("Loading ASCAT data...")
    paths = yaml.safe_load(Path("../paths.yml").read_text())
    system_key = "windows" if platform.system() == "Windows" else "linux"
    root = Path(paths[system_key]).expanduser()
    cell_source = root / "datasets/scat_ard/ascat_ssm_cdr_12.5km_h121"
    assert cell_source.exists(), f"ASCAT data path does not exist: {cell_source}"

    h121_reader = CellGridFiles.from_product_id(cell_source, "H121")
    ascat_ds = h121_reader.read(bbox=bbox)
    print("Done")

    # Reprojecting ASCAT points to Sentinel CRS
    print("Processing...")
    points_ds = ascat_ds.cf_geom.to_point_array()
    x, y = Transformer.from_crs("EPSG:4326", sentinel_epsg, always_xy=True).transform(points_ds.lon, points_ds.lat)
    points_ds['e7x'] = ('obs', x)
    points_ds['e7y'] = ('obs', y)

    # Collocating Sentinel-1 with ASCAT using nearest-neighbor
    ascat_sentinel = sentinel_ds.sel(
        x=points_ds.e7x,
        y=points_ds.e7y,
        time=points_ds.time,
        method="nearest"
    )

    # Add Sentinel backscatter bands to ASCAT dataset
    ascat_ds['s1-VH'] = ("obs", ascat_sentinel['VH'].data)
    ascat_ds['s1-VV'] = ("obs", ascat_sentinel['VV'].data)
    print("Done")

    # Save with progress bar
    ascat_ds['subsurface_scattering_probability'].encoding.update({'dtype': 'float32'}) # Ensure correct dtype as NANs are a problem otherwise
    print(f"Saving output to {output_path}...")
    with ProgressBar():
        ascat_ds.to_netcdf(output_path)
    print(f"Done! Collocated dataset saved to {output_path}")


In [31]:
process_sentinel_and_ascat(
    zarr_path="output_6km_filled.zarr",
    sentinel_epsg="EPSG:27704")

Loading Sentinel-1 data...
Done
Loading ASCAT data...
Done
Processing...
Done
Saving output to ascat_s1_collocated.nc...
[########################################] | 100% Completed | 59.82 s
Done! Collocated dataset saved to ascat_s1_collocated.nc


## Testing the Workflow

### Loading Sentinel 1 data

In [8]:
sentinel_ds = xr.open_zarr("output_6km_filled.zarr")
sentinel_ds

<xarray.Dataset> Size: 11kB
Dimensions:  (time: 2, y: 26, x: 26)
Coordinates:
  * time     (time) datetime64[ns] 16B 2022-01-06T19:00:00 2022-01-07T07:00:00
  * x        (x) float64 208B 3.143e+06 3.15e+06 ... 3.293e+06 3.298e+06
  * y        (y) float64 208B 8.969e+05 8.906e+05 ... 7.346e+05 7.267e+05
Data variables:
    VH       (time, y, x) float32 5kB dask.array<chunksize=(1, 25, 25), meta=np.ndarray>
    VV       (time, y, x) float32 5kB dask.array<chunksize=(1, 25, 25), meta=np.ndarray>

In [11]:
# get bounds for ASCAT data from S1

# is this a good approach? look at step 1 to see why

sentinel_ds = sentinel_ds.rio.write_crs("EPSG:27704")

#bounds_proj = sentinel_ds.rio.bounds()

sentinel_ds_latlon = sentinel_ds.rio.reproject("EPSG:4326")

latmin = float(sentinel_ds_latlon.y.min())
latmax = float(sentinel_ds_latlon.y.max())
lonmin = float(sentinel_ds_latlon.x.min())
lonmax = float(sentinel_ds_latlon.x.max())

bbox=(latmin, latmax, lonmin, lonmax)

print(f"bbox: {bbox}")

bbox: (35.677603076219796, 37.581347591980396, -6.664028656833048, -4.420329763258051)


c:\Users\jwagner\AppData\Local\anaconda3\envs\ascat-s1-synergy\lib\site-packages\dask\array\chunk.py:279: RuntimeWarning: invalid value encountered in cast
  return x.astype(astype_dtype, **kwargs)


We can see that we do not get the exact defined lat and lon we defined but a slightly bigger area. This should not be a problem in our further analysis.

### Reading ASCAT data points

In [12]:
# read yaml for reading data on Windows, Linux or the JupyterHub
paths = yaml.safe_load(Path("../paths.yml").read_text())
root = Path(paths["windows" if platform.system() == "Windows" else "linux"]).expanduser()
cell_source = root / "datasets/scat_ard/ascat_ssm_cdr_12.5km_h121"
assert cell_source.exists(), f"Path does not exist: {cell_source}"

Documentation from:
https://github.com/TUW-GEO/ascat/blob/master/docs/org/reading_cells.org

In [ ]:
h121_reader = CellGridFiles.from_product_id(cell_source, "H121")

ascat_ds = h121_reader.read(bbox=bbox)
ascat_ds

<xarray.Dataset> Size: 219MB
Dimensions:                            (obs: 4206310, locations: 274)
Coordinates:
    time                               (obs) datetime64[ns] 34MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    lon                                (locations) float32 1kB dask.array<chunksize=(274,), meta=np.ndarray>
    lat                                (locations) float32 1kB dask.array<chunksize=(274,), meta=np.ndarray>
    alt                                (locations) float32 1kB dask.array<chunksize=(274,), meta=np.ndarray>
Dimensions without coordinates: obs, locations
Data variables: (12/21)
    as_des_pass                        (obs) int8 4MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    swath_indicator                    (obs) int8 4MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    surface_flag                       (obs) uint8 4MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    surface_soil_moisture              (obs) float32 17MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    surface_soil_moisture_noise        (obs) float32 17MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    backscatter40                      (obs) float32 17MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    ...                                 ...
    topographic_complexity             (obs) int8 4MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    subsurface_scattering_probability  (obs) float64 34MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    sat_id                             (obs) int8 4MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    row_size                           (locations) int64 2kB dask.array<chunksize=(274,), meta=np.ndarray>
    location_id                        (locations) int64 2kB dask.array<chunksize=(274,), meta=np.ndarray>
    location_description               (locations) <U1 1kB dask.array<chunksize=(274,), meta=np.ndarray>
Attributes:
    id:                 1249.nc
    date_created:       2025-01-09 12:30:17
    featureType:        timeSeries
    grid_mapping_name:  fibgrid_12.5

In [14]:
points_ds = ascat_ds.cf_geom.to_point_array()
points_ds

<xarray.Dataset> Size: 320MB
Dimensions:                            (obs: 4206310)
Coordinates:
    time                               (obs) datetime64[ns] 34MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    lon                                (obs) float32 17MB -6.244 ... -4.64
    lat                                (obs) float32 17MB 35.68 35.68 ... 37.55
    alt                                (obs) float32 17MB 9.969e+36 ... 9.969...
Dimensions without coordinates: obs
Data variables: (12/20)
    as_des_pass                        (obs) int8 4MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    swath_indicator                    (obs) int8 4MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    surface_flag                       (obs) uint8 4MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    surface_soil_moisture              (obs) float32 17MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    surface_soil_moisture_noise        (obs) float32 17MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    backscatter40                      (obs) float32 17MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    ...                                 ...
    wetland_fraction                   (obs) int8 4MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    topographic_complexity             (obs) int8 4MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    subsurface_scattering_probability  (obs) float64 34MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    sat_id                             (obs) int8 4MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    location_id                        (obs) int64 34MB 958101 ... 1001364
    location_description               (obs) <U1 17MB '' '' '' '' ... '' '' ''
Attributes:
    id:                 1249.nc
    date_created:       2025-01-09 12:30:17
    featureType:        point
    grid_mapping_name:  fibgrid_12.5

In [15]:
from pyproj import Transformer
x, y = Transformer.from_crs("EPSG:4326", "EPSG:27704", always_xy=True).transform(points_ds.lon, points_ds.lat)
x

array([3122906.75177061, 3122906.75177061, 3122906.75177061, ...,
       3331201.895713  , 3331201.895713  , 3331201.895713  ])

In [16]:
points_ds['e7x'] = ('obs', x)
points_ds['e7y'] = ('obs', y)

In [17]:
points_ds

<xarray.Dataset> Size: 387MB
Dimensions:                            (obs: 4206310)
Coordinates:
    time                               (obs) datetime64[ns] 34MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    lon                                (obs) float32 17MB -6.244 ... -4.64
    lat                                (obs) float32 17MB 35.68 35.68 ... 37.55
    alt                                (obs) float32 17MB 9.969e+36 ... 9.969...
Dimensions without coordinates: obs
Data variables: (12/22)
    as_des_pass                        (obs) int8 4MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    swath_indicator                    (obs) int8 4MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    surface_flag                       (obs) uint8 4MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    surface_soil_moisture              (obs) float32 17MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    surface_soil_moisture_noise        (obs) float32 17MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    backscatter40                      (obs) float32 17MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    ...                                 ...
    subsurface_scattering_probability  (obs) float64 34MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    sat_id                             (obs) int8 4MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    location_id                        (obs) int64 34MB 958101 ... 1001364
    location_description               (obs) <U1 17MB '' '' '' '' ... '' '' ''
    e7x                                (obs) float64 34MB 3.123e+06 ... 3.331...
    e7y                                (obs) float64 34MB 7.377e+05 ... 8.804...
Attributes:
    id:                 1249.nc
    date_created:       2025-01-09 12:30:17
    featureType:        point
    grid_mapping_name:  fibgrid_12.5

In [18]:
ascat_sentinel = sentinel_ds.sel(x=points_ds.e7x, y=points_ds.e7y, time=points_ds.time, method="nearest")
ascat_sentinel

<xarray.Dataset> Size: 185MB
Dimensions:      (obs: 4206310)
Coordinates:
    time         (obs) datetime64[ns] 34MB 2022-01-06T19:00:00 ... 2022-01-07...
    x            (obs) float64 34MB 3.143e+06 3.143e+06 ... 3.298e+06 3.298e+06
    y            (obs) float64 34MB 7.346e+05 7.346e+05 ... 8.782e+05 8.782e+05
    spatial_ref  int32 4B 0
    lon          (obs) float32 17MB -6.244 -6.244 -6.244 ... -4.64 -4.64 -4.64
    lat          (obs) float32 17MB 35.68 35.68 35.68 ... 37.55 37.55 37.55
    alt          (obs) float32 17MB 9.969e+36 9.969e+36 ... 9.969e+36 9.969e+36
Dimensions without coordinates: obs
Data variables:
    VH           (obs) float32 17MB dask.array<chunksize=(625,), meta=np.ndarray>
    VV           (obs) float32 17MB dask.array<chunksize=(625,), meta=np.ndarray>

In [19]:
ascat_ds['s1-VH'] = ("obs", ascat_sentinel['VH'].data)
ascat_ds['s1-VV'] = ("obs", ascat_sentinel['VV'].data)
ascat_ds

<xarray.Dataset> Size: 252MB
Dimensions:                            (obs: 4206310, locations: 274)
Coordinates:
    time                               (obs) datetime64[ns] 34MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    lon                                (locations) float32 1kB dask.array<chunksize=(274,), meta=np.ndarray>
    lat                                (locations) float32 1kB dask.array<chunksize=(274,), meta=np.ndarray>
    alt                                (locations) float32 1kB dask.array<chunksize=(274,), meta=np.ndarray>
Dimensions without coordinates: obs, locations
Data variables: (12/23)
    as_des_pass                        (obs) int8 4MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    swath_indicator                    (obs) int8 4MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    surface_flag                       (obs) uint8 4MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    surface_soil_moisture              (obs) float32 17MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    surface_soil_moisture_noise        (obs) float32 17MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    backscatter40                      (obs) float32 17MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    ...                                 ...
    sat_id                             (obs) int8 4MB dask.array<chunksize=(1000000,), meta=np.ndarray>
    row_size                           (locations) int64 2kB dask.array<chunksize=(274,), meta=np.ndarray>
    location_id                        (locations) int64 2kB dask.array<chunksize=(274,), meta=np.ndarray>
    location_description               (locations) <U1 1kB dask.array<chunksize=(274,), meta=np.ndarray>
    s1-VH                              (obs) float32 17MB dask.array<chunksize=(625,), meta=np.ndarray>
    s1-VV                              (obs) float32 17MB dask.array<chunksize=(625,), meta=np.ndarray>
Attributes:
    id:                 1249.nc
    date_created:       2025-01-09 12:30:17
    featureType:        timeSeries
    grid_mapping_name:  fibgrid_12.5